In [1]:
from backtesting import set_bokeh_output
set_bokeh_output(notebook=False)

# !python3 -m pip install bokeh==2.4.3

In [2]:
import sys
from datetime import timedelta

In [3]:
# sys.path.insert(1, '/home/kenny/algotrading/model_training')
sys.path.insert(1, "/Users/arsenchik/Desktop/dipploma/machine_learning_in_hft/algotrading/src/backtesting")
sys.path.insert(1, '/Users/arsenchik/Desktop/dipploma/machine_learning_in_hft/algotrading/src/models')
sys.path.insert(1, '/Users/arsenchik/Desktop/dipploma/machine_learning_in_hft/algotrading/src/configs')
sys.path.insert(1, "/Users/arsenchik/Desktop/dipploma/machine_learning_in_hft/algotrading/src/data")

In [4]:
import numpy as np
import pandas as pd
import lightgbm as lgb
from tqdm import tqdm
from backtesting import Backtest, Strategy
from sqlalchemy import create_engine, text
import pandas_ta as pta
from backtesting.lib import crossover

from strategies import MultipleBasic, SmaCross, BinaryBasic, Multiple, Binary, Random
from config import *
from dataset import Data
from experiments import EXPERIMENT_ID_TO_FEATURES

In [5]:
url = DATABASE_URL_1MINUTE
td_days = 100

In [6]:
engine = create_engine(url=url)

with engine.connect() as connection:
        # query = "SELECT name FROM sqlite_schema WHERE type='table'"
        # symbols = [item[0] for item in connection.execute(text(query)).fetchall()]
        # for symbol in symbols:
        symbol = 'BTCUSDT'
        qry = f"SELECT * FROM '{symbol}' WHERE Open_Time < '{pd.to_datetime('today') - timedelta(days = td_days)}'"
        data = pd.DataFrame(connection.execute(text(qry)))
        data['Open_Time'] = pd.to_datetime(data['Open_Time'])
        data.set_index("Open_Time", inplace=True)
        # data.index = pd.to_datetime(data.index)
        # data.drop(['Close_Time'], axis=1, inplace=True)
connection.close()

In [7]:
data

,Open,High,Low,Close,Volume
Open_Time,,,,,
2023-02-22 06:08:00,24101.42,24108.72,24092.42,24093.79,153.14142
2023-02-22 06:09:00,24092.79,24107.48,24090.28,24106.89,123.11680
2023-02-22 06:10:00,24106.15,24109.69,24093.49,24093.99,79.38456
2023-02-22 06:11:00,24095.00,24103.93,24092.78,24102.82,88.00198
2023-02-22 06:12:00,24102.82,24110.71,24082.57,24084.94,296.63354
...,...,...,...,...,...
2023-02-27 06:03:00,23400.53,23408.17,23388.30,23399.61,232.82946
2023-02-27 06:04:00,23398.94,23400.47,23380.33,23391.52,170.03786
2023-02-27 06:05:00,23391.52,23394.39,23380.41,23387.48,163.53791


In [12]:
data[data.index < data.index[0] + pd.Timedelta(days=2)]

,Open,High,Low,Close,Volume
Open_Time,,,,,
2023-02-22 06:08:00,24101.42,24108.72,24092.42,24093.79,153.14142
2023-02-22 06:09:00,24092.79,24107.48,24090.28,24106.89,123.11680
2023-02-22 06:10:00,24106.15,24109.69,24093.49,24093.99,79.38456
2023-02-22 06:11:00,24095.00,24103.93,24092.78,24102.82,88.00198
2023-02-22 06:12:00,24102.82,24110.71,24082.57,24084.94,296.63354
...,...,...,...,...,...
2023-02-24 06:03:00,23880.78,23884.98,23878.17,23882.70,116.79325
2023-02-24 06:04:00,23883.41,23885.10,23878.12,23879.93,53.21691
2023-02-24 06:05:00,23879.93,23884.26,23876.92,23881.51,93.56253


In [ ]:
import lightgbm as lgb
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score

def _slice_data(df: pd.DataFrame, timeframe: int, index_id: int) -> pd.DataFrame:
    return df[df.index < df.index[index_id] + pd.Timedelta(days=timeframe)]


def _training(df: pd.DataFrame):
    model = lgb.Class

In [7]:
# data['RSI'] = pta.rsi(data['Close'], length=21)
# data['EMA_6'] = pta.ema(data['Close'], length=6)
# data['EMA_22'] = pta.ema(data['Close'], length=22)
# data['EMA_127'] = pta.ema(data['Close'], length=127)

In [8]:
def _get_ema(close, index, length):
    df = pd.DataFrame({"data": list(index), "close": list(close)}).set_index("data")
    return list(pta.ema(df.close, length=length))

def _get_rsi(close, index, length):
    df = pd.DataFrame(({"data": list(index), "close": list(close)})).set_index("data")
    return list(pta.rsi(df.close, length=length))

def _get_macd_main_line(close, index, fast, slow, signal):
    df = pd.DataFrame({"data": list(index), "close": list(close)}).set_index("data")
    return list(pta.macd(df.close, fast=fast, slow=slow, signal=signal)["MACD_12_26_9"])

def _get_macd_signal_line(close, index, fast, slow, signal):
    df = pd.DataFrame({"data": list(index), "close": list(close)}).set_index("data")
    return list(pta.macd(df.close, fast=fast, slow=slow, signal=signal)["MACDs_12_26_9"])

def _get_macd_hist(close, index, fast, slow, signal):
    df = pd.DataFrame({"data": list(index), "close": list(close)}).set_index("data")
    return list(pta.macd(df.close, fast=fast, slow=slow, signal=signal)["MACDh_12_26_9"])


In [9]:
# class MR(Strategy):


#     def init(self):
#         self.main_line = self.I(_get_macd_main_line, self.data.Close, self.data.index, 12,26,9,name="main_line")
#         self.signal_line = self.I(_get_macd_signal_line, self.data.Close, self.data.index, 12,26,9, name="signal_line")
#         self.histogram = self.I(_get_macd_hist, self.data.Close, self.data.index, 12,26,9, name="histogram")
#         self.rsi = self.I(_get_rsi, self.data.Close, self.data.index, 21, name="RSI")

#     def next(self):
#         _close = self.data.Close[-1]
#         _high = self.data.High[-1]
#         _low = self.data.Low[-1]
#         _open = self.data.Open[-1]

#         price = (_close + _open + _high + _low) / 4

#         main_line_value = self.main_line[-1]
#         signal_line_value = self.signal_line[-1]
#         hist_value = self.histogram[-1]
#         hist_value_previous = self.histogram[-2]

#         rsi_value = self.rsi[-1]

#         if crossover(main_line_value, signal_line_value) and  rsi_value < 30:
#             if self.position.is_long or not self.position:
#                 self.buy()
#             elif self.position.is_short:
#                 self.position.close()
#                 self.buy()
#         elif crossover(signal_line_value, main_line_value) and rsi_value > 70:
#             if self.position.is_short or not self.position:
#                 self.sell()
#             elif self.position.is_long:
#                 self.position.close()
#                 self.sell()
        
#         if hist_value < hist_value_previous and self.position.is_short:
#             self.position.close()
#         elif hist_value > hist_value_previous and self.position.is_long:
#             self.position.close()



In [10]:
# deposit = 100000
# commission = 0.0004

# bt_MR = Backtest(data, MR, cash=deposit, commission=commission)

In [11]:
# bt_MR.run()

In [12]:
# bt_MR.plot()

In [48]:
class TrafficLight1(Strategy):
    def init(self):
        self.close = self.data.Close
        self.high = self.data.High
        self.low = self.data.Low
        
        # green -> yellow -> red
        # red -> yellow -> green
        self.green = self.I(_get_ema, self.data.Close, self.data.index, 6,name="EMA_6", color="green")
        self.yellow = self.I(_get_ema, self.data.Close, self.data.index, 22, name="EMA_22", color="yellow")
        self.red = self.I(_get_ema, self.data.Close, self.data.index, 127, name="EMA_127", color="red")
        

    def next(self):
        current_time = self.data.index[-1]
        price = self.data.Close[-1]
        high = self.data.High[-1]
        low = self.data.Low[-1]
        red = self.red[-1]
        yellow = self.yellow[-1]
        green = self.green[-1]
        if red > yellow and yellow > green:
            if (high >= red or high >= yellow) and (not self.position.is_short):
                self.sell(size=0.15, sl=price * 1.003, tp=price * 0.994)
                # self.sell(size=0.15)
        elif red < yellow and yellow < green:
            if (low <= red or low <= yellow) and (not self.position.is_long):
                self.buy(size=0.15, sl=price * 0.997, tp=price * 1.006)
                # self.buy(size=0.15)
        
        for trade in self.trades:
            if current_time - trade.entry_time > pd.Timedelta(minutes=1):
                # if trade.is_long:
                #     trade.sl = max(trade.sl, low)
                # else:
                #     trade.sl = min(trade.sl, high)
                self.position.close()

In [40]:
data["EMA_6"] = _get_ema(data.Close, data.index, length=6)
data["EMA_22"] = _get_ema(data.Close, data.index, length=22)
data["EMA_127"] = _get_ema(data.Close, data.index, length=127)

In [43]:
data[["EMA_6", "EMA_22", "EMA_127"]].describe()

,EMA_6,EMA_22,EMA_127
count,7195.000000,7179.000000,7074.000000
mean,23580.008508,23579.675427,23577.180429
std,453.633336,452.913501,448.544645
min,22850.639091,22889.956942,22925.280256
25%,23149.469098,23150.973881,23154.324765
50%,23564.157610,23566.459257,23536.971260
75%,23952.576843,23955.982243,23953.793130
max,24570.383415,24543.483643,24468.483267


In [101]:
class TrafficLight2(Strategy):
    def init(self):
        self.close = self.data.Close
        self.high = self.data.High
        self.low = self.data.Low
        self.depth = 20
        # green -> yellow -> red
        # red -> yellow -> green
        self.green = self.I(_get_ema, self.data.Close, self.data.index, 6,name="EMA_6", color="green")
        self.yellow = self.I(_get_ema, self.data.Close, self.data.index, 22, name="EMA_22", color="yellow")
        self.red = self.I(_get_ema, self.data.Close, self.data.index, 127, name="EMA_127", color="red")
        self.not_first_short = False
        self.not_first_long = False

    def next(self):
        current_time = self.data.index[-1]
        price = self.data.Close[-1]
        high = self.data.High[-1]
        low = self.data.Low[-1]
        red = self.red[-1]
        yellow = self.yellow[-1]
        green = self.green[-1]
        # if red >= yellow + self.depth and yellow >= green + self.depth and not self.not_first_short:
        if red >= yellow and yellow > green:
            # if (high >= red or high >= yellow) and (not self.position.is_short):
            #     self.sell(size=0.5, sl=price * 1.006, tp=price * 0.997)
            if (high >= red) and (not self.position.is_short):
                self.sell(size=0.15, sl=price*1.005, tp=price*0.99)
                self.not_first_short = True
                self.not_first_long = False
                self.position.close()
        # elif red + self.depth <= yellow and yellow + self.depth <= green and not self.not_first_long:
        elif red < yellow and yellow < green:
            # if (low <= red or low <= yellow) and (not self.position.is_long):
            #     self.buy(size=0.5, sl=price * 0.997, tp=price * 1.006)
            if (low <= red) and (not self.position.is_long):
                self.position.close()
                self.buy(size=0.15, sl=price * 0.995, tp=price * 1.015)
                self.not_first_long = True
                self.not_first_short = False
        
        for trade in self.trades:
            if current_time - trade.entry_time > pd.Timedelta(minutes=2):
                # if trade.is_long:
                #     trade.sl = max(trade.sl, low)
                # else:
                #     trade.sl = min(trade.sl, high)
                self.position.close()

In [102]:
cash = 100000
commission = .0004
bt_random = Backtest(data, TrafficLight2, cash=cash, commission=commission, margin=0.2, trade_on_close=True)
output = bt_random.run()

In [103]:
output

Start                     2023-02-22 06:08:00
End                       2023-02-27 06:07:00
Duration                      4 days 23:59:00
Exposure Time [%]                    6.722222
Equity Final [$]                 96286.543888
Equity Peak [$]                 100215.627748
Return [%]                          -3.713456
Buy & Hold Return [%]               -2.963834
Return (Ann.) [%]                  -84.400481
Volatility (Ann.) [%]                1.564926
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -3.993104
Avg. Drawdown [%]                   -1.381341
Max. Drawdown Duration        4 days 16:55:00
Avg. Drawdown Duration        1 days 14:11:00
# Trades                                  124
Win Rate [%]                        25.806452
Best Trade [%]                       0.346553
Worst Trade [%]                     -0.540216
Avg. Trade [%]                    

In [104]:
output["_trades"]

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-3,134,137,24013.070928,24014.33,-3.777216,-0.000052,2023-02-22 08:22:00,2023-02-22 08:25:00,0 days 00:03:00
1,-3,139,142,23993.718672,23980.63,39.266016,0.000546,2023-02-22 08:27:00,2023-02-22 08:30:00,0 days 00:03:00
2,-3,335,338,24105.304020,24127.33,-66.077940,-0.000914,2023-02-22 11:43:00,2023-02-22 11:46:00,0 days 00:03:00
3,-3,410,413,24122.956956,24131.59,-25.899132,-0.000358,2023-02-22 12:58:00,2023-02-22 13:01:00,0 days 00:03:00
4,-3,422,425,24052.025340,23995.01,171.046020,0.002371,2023-02-22 13:10:00,2023-02-22 13:13:00,0 days 00:03:00
...,...,...,...,...,...,...,...,...,...,...
119,3,6793,6796,23523.065464,23515.73,-22.006392,-0.000312,2023-02-26 23:21:00,2023-02-26 23:24:00,0 days 00:03:00
120,3,6804,6807,23545.214320,23558.86,40.937040,0.000580,2023-02-26 23:32:00,2023-02-26 23:35:00,0 days 00:03:00
121,-3,6849,6852,23507.983044,23522.10,-42.350868,-0.000601,2023-02-27 00:17:00,2023-02-27 00:20:00,0 days 00:03:00
122,-3,6853,6856,23504.154576,23481.90,66.763728,0.000947,2023-02-27 00:21:00,2023-02-27 00:24:00,0 days 00:03:00


In [105]:
bt_random.plot()

Row(id='16632', ...)

## Random classifier

In [7]:
cash = 100000
commission = .0001
bt_random = Backtest(data, Random, cash=cash, commission=commission)
output = bt_random.run()

In [8]:
output

Start                     2023-02-22 06:08:00
End                       2023-02-27 06:07:00
Duration                      4 days 23:59:00
Exposure Time [%]                   77.791667
Equity Final [$]                 92546.902942
Equity Peak [$]                      100000.0
Return [%]                          -7.453097
Buy & Hold Return [%]               -2.963834
Return (Ann.) [%]                  -98.126239
Volatility (Ann.) [%]                0.408417
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -7.490269
Avg. Drawdown [%]                   -7.490269
Max. Drawdown Duration        4 days 23:58:00
Avg. Drawdown Duration        4 days 23:58:00
# Trades                                 2759
Win Rate [%]                        41.536789
Best Trade [%]                       0.543709
Worst Trade [%]                     -1.089354
Avg. Trade [%]                    

In [9]:
bt_random.plot()

Row(id='1475', ...)

## Binary LightCoin strategy

In [5]:
binary_model_path_txt = '/Users/arsenchik/Desktop/dipploma/machine_learning_in_hft/algotrading/model_training/models/binary_model_1_min.txt'

binary_model = lgb.Booster(model_file=binary_model_path_txt)

features = EXPERIMENT_ID_TO_FEATURES[24]
test_data = Data(data)
test_data.create_features()
test_data.df['predicted_proba'] = binary_model.predict(test_data.df[features])

In [6]:
cash = 100000
commission = .0001

bt = Backtest(test_data.df, Binary, cash=cash, commission=commission)
output = bt.run()
# stats = bt.optimize(price_delta=[0.002, 0.004, 0.001],
#                   maximize='Sharpe Ratio')

In [7]:
output

Start                     2023-02-22 06:08:00
End                       2023-02-27 06:07:00
Duration                      4 days 23:59:00
Exposure Time [%]                   83.861111
Equity Final [$]                100636.898432
Equity Peak [$]                 101267.404378
Return [%]                           0.636898
Buy & Hold Return [%]               -2.963834
Return (Ann.) [%]                   54.337715
Volatility (Ann.) [%]               12.011848
Sharpe Ratio                         4.523676
Sortino Ratio                       18.265706
Calmar Ratio                        54.923605
Max. Drawdown [%]                   -0.989333
Avg. Drawdown [%]                   -0.130465
Max. Drawdown Duration        1 days 22:31:00
Avg. Drawdown Duration        0 days 04:17:00
# Trades                                   27
Win Rate [%]                        55.555556
Best Trade [%]                       3.546073
Worst Trade [%]                     -1.336301
Avg. Trade [%]                    

In [8]:
bt.plot()

Row(id='1474', ...)

## Multiple LigthCoin strategy

In [5]:
cash = 100000
commission = .0001

bt = Backtest(data, Multiple, cash=cash, commission=commission)
output = bt.run()
# stats = bt.optimize(price_delta=[0.002, 0.004, 0.001],
#                   maximize='Sharpe Ratio')

In [6]:
output

Start                     2023-02-22 06:08:00
End                       2023-02-27 06:07:00
Duration                      4 days 23:59:00
Exposure Time [%]                   32.083333
Equity Final [$]                100519.904699
Equity Peak [$]                 100701.760357
Return [%]                           0.519905
Buy & Hold Return [%]               -2.963834
Return (Ann.) [%]                   30.263239
Volatility (Ann.) [%]               10.201341
Sharpe Ratio                         2.966594
Sortino Ratio                        11.09004
Calmar Ratio                        37.399474
Max. Drawdown [%]                   -0.809189
Avg. Drawdown [%]                   -0.156033
Max. Drawdown Duration        2 days 14:03:00
Avg. Drawdown Duration        0 days 07:58:00
# Trades                                   28
Win Rate [%]                        35.714286
Best Trade [%]                       2.769027
Worst Trade [%]                     -0.304868
Avg. Trade [%]                    

In [7]:
bt.plot()

Row(id='1475', ...)

In [5]:
cash = 100000
commission = .0001
bt = Backtest(data, Binary, cash=cash, commission=commission)
output = bt.run()

In [6]:
output

Start                     2023-02-22 06:08:00
End                       2023-02-27 06:07:00
Duration                      4 days 23:59:00
Exposure Time [%]                    0.097222
Equity Final [$]                100007.827219
Equity Peak [$]                 100007.827219
Return [%]                           0.007827
Buy & Hold Return [%]               -2.963834
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     0.0
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                   -0.018607
Avg. Drawdown [%]                   -0.018607
Max. Drawdown Duration        0 days 00:08:00
Avg. Drawdown Duration        0 days 00:08:00
# Trades                                    7
Win Rate [%]                        71.428571
Best Trade [%]                       0.025829
Worst Trade [%]                     -0.040295
Avg. Trade [%]                    

In [7]:
bt.plot()

Row(id='1474', ...)

In [8]:
output

Start                     2023-02-22 06:08:00
End                       2023-02-27 06:07:00
Duration                      4 days 23:59:00
Exposure Time [%]                    9.194444
Equity Final [$]                 98472.648418
Equity Peak [$]                      100000.0
Return [%]                          -1.527352
Buy & Hold Return [%]               -2.963834
Return (Ann.) [%]                  -50.102855
Volatility (Ann.) [%]                  0.9343
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -1.527352
Avg. Drawdown [%]                   -1.527352
Max. Drawdown Duration        4 days 23:46:00
Avg. Drawdown Duration        4 days 23:46:00
# Trades                                  432
Win Rate [%]                        44.907407
Best Trade [%]                       0.049496
Worst Trade [%]                     -0.067931
Avg. Trade [%]                    

In [8]:
output

Start                     2023-02-22 06:08:00
End                       2023-02-27 06:07:00
Duration                      4 days 23:59:00
Exposure Time [%]                   62.291667
Equity Final [$]                 92527.550636
Equity Peak [$]                 100102.361703
Return [%]                          -7.472449
Buy & Hold Return [%]               -2.963834
Return (Ann.) [%]                  -94.390044
Volatility (Ann.) [%]                0.820879
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -7.567065
Avg. Drawdown [%]                   -1.526705
Max. Drawdown Duration        4 days 23:32:00
Avg. Drawdown Duration        0 days 23:58:00
# Trades                                 2247
Win Rate [%]                        46.283934
Best Trade [%]                        0.07142
Worst Trade [%]                     -0.096647
Avg. Trade [%]                    

In [9]:
bt.plot()

Row(id='1475', ...)